# Generate ER Diagrams from Semantic link

Run on Fabric Spark only.
Update "semantic_model_name" with your semantic model name

### Set up

In [1]:
%pip install semantic-link

StatementMeta(, 02c58d86-ad08-459a-8c69-695ca6b167b0, 7, Submitted, Running)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 140.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.4/932.4 kB 145.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 171.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 133.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 105.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 119.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 28.1 MB/s eta 0:00:00
     ━

In [ ]:
semantic_model_name = 'D365 Reporting Datawarehouse'

StatementMeta(, , , Waiting, )

### Get Relationships from Semantic link

In [ ]:
import sempy.fabric as fabric
from sempy.relationships import plot_relationship_metadata
df_relationships = fabric.list_relationships(semantic_model_name)

StatementMeta(, , , Waiting, )

In [ ]:
plot_relationship_metadata(df_relationships)

StatementMeta(, , , Waiting, )

### Generate Mermaid code

In [ ]:
# Function to convert Multiplicity to Mermaid notation
def multiplicity_to_mermaid(multiplicity, direction):

    if(direction == 'l'):
        switcher = {
            '0': '|o',
            '1': '||',
            'm': '}|'
        }
    else:
        switcher = {
            '0': 'o|',
            '1': '||',
            'm': '|{'
        }
    return switcher.get(multiplicity, '')  # Return an empty string for unknown multiplicities


# Function to convert DataFrame rows to Mermaid syntax with multiplicity
def row_to_mermaid(row):
    multiplicity = row['Multiplicity'].split(':')
    from_multiplicity = multiplicity_to_mermaid(multiplicity[0], 'l')
    to_multiplicity = multiplicity_to_mermaid(multiplicity[1],'r')
    return f"    {row['From Table']} {from_multiplicity}--{to_multiplicity} {row['To Table']} : {row['From Column']}"

# Convert each row of the DataFrame to Mermaid syntax

title = f'''---
title: {semantic_model_name} ER Diagram
config:
  theme: dark
  er:
    layoutDirection: RL
---'''


mermaid_code =  "\n".join(df_relationships.apply(row_to_mermaid, axis=1))

# Generate the complete Mermaid code
mermaid_code = f"{title}\nerDiagram\n{mermaid_code}"

# Print or use the generated Mermaid code
print(mermaid_code)

StatementMeta(, , , Waiting, )

### Generate dbdiagrams.io code

In [ ]:
import sempy
import sempy.fabric as fabric
from pyspark.sql.functions import col, concat, lit

# Get columns from semantic link
fabric_df = fabric.list_columns(semantic_model_name)

# Filter the dataframe for tables that start with a Dim or Fact only
dw_tables = spark.createDataFrame(fabric_df).filter((col("Table Name").startswith("Dim")) | (col("Table Name").startswith("Fact")))

# Convert relationships to a spark dataframe
dw_relationships = spark.createDataFrame(df_relationships)

# Get a list of all tables and columns by unioning the from table & column with the to table & column
table_column_df = dw_relationships.select("From Table", "From Column").union(dw_relationships.select("To Table", "To Column")).dropDuplicates()

# Join the columns together & Drop the original "From Table" and "From Column" columns
relationship_columns = table_column_df.withColumn("Table_Column", concat(table_column_df["From Table"], lit("_"), table_column_df["From Column"])) \
                                       .drop("From Table", "From Column")

# Extracting unique values from the 'Table_Column' column and storing them in a set
relationship_columns = set(relationship_columns.select("Table_Column").rdd.flatMap(lambda x: x).collect())

StatementMeta(, , , Waiting, )

In [ ]:
# Extract unique table names from the DataFrame 'dw_tables' and storing them as a list
table_names = dw_tables.select("Table Name").distinct().rdd.map(lambda x: x[0]).collect()

# Function to generate DDL for a table
def generate_ddl(table_name):
    # Filter DataFrame for the specified table
    table_df = dw_tables.filter(dw_tables["Table Name"] == table_name)

    # Extract column information
    columns_ddl = []

    for row in table_df.collect():
        table_column_name = f"{row['Table Name']}_{row['Column Name']}"  # Combine table name and column name

        if table_column_name in relationship_columns:
            column_ddl = f"{row['Column Name']} {row['Data Type']}"
            if row['Key']:
                column_ddl += " [primary key]"
            columns_ddl.append(column_ddl)

    # Exclude tables with no relationships
    if len(columns_ddl) == 0:
        return None  # Return None if no columns are found

    # Generate DDL for the table
    ddl = f"Table {table_name} {{\n    "
    ddl += "\n    ".join(columns_ddl)
    ddl += "\n}\n"  

    return ddl

# Generate DDL for all tables
all_ddl = [generate_ddl(table_name) for table_name in table_names]

# Generate dbdiagrams.io relationship
def multiplicity_to_dbdigramsio(multiplicity):
    switcher = {
        'm:1': '>',
        '1:m': '<',
        '1:1': '--',
        'm:m': '<>'
    }
    return switcher.get(multiplicity, '')  # Return an empty string for unknown multiplicities


# generate dbdiagram relationship
def row_to_dbdiagram_relationship(row):  
    return f"Ref:  {row['From Table']}.{row['From Column']} {multiplicity_to_dbdigramsio(row['Multiplicity'])} {row['To Table']}.{row['To Column']}"

# Generate diagrams.io line code
relationship_dll =  "\n".join(df_relationships.apply(row_to_dbdiagram_relationship, axis=1))

# Print DDL for all tables
for ddl in all_ddl:
    if ddl is not None:
        print(ddl)

print (relationship_dll)

StatementMeta(, , , Waiting, )